In [1]:
import pandas as pd
import pickle

In [2]:
dataset=pd.read_csv('prep.csv')
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,False,False,False,False,False,False,True,True,False,True
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,True,False,False,False,False,False,True,False,False,True
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,True,False,False,False,False,False,True,False,False,True
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,True,False,False,False,False,False,True,False,True,True
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,True,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,True,False,False,False,False,False,True,False,False,True
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,True,False,False,True,True,False,True,False,True,True
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,True,False,False,True,True,False,False,False,False,True
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,True,False,False,True,True,False,True,False,True,True


In [3]:
independent=dataset.drop('classification_yes',axis=1)
dependent=dataset['classification_yes']

In [10]:
from sklearn.feature_selection import SelectKBest, chi2
def selectk(independent,dependent,n):
    test=SelectKBest(score_func=chi2,k=n)
    fit1=test.fit(independent,dependent)
    selectk=fit1.transform(independent)
    return selectk

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
def split_scaler(independent,dependent):
    X_train,X_test,Y_train,Y_test=train_test_split(independent,dependent,test_size=0.25,random_state=0)
    sc=StandardScaler()
    X_train=sc.fit_transform(X_train)
    X_test=sc.transform(X_test)
    return X_train,X_test,Y_train,Y_test

In [6]:
def r2_prediction(regressor,X_test,Y_test):
    Y_pred=regressor.predict(X_test)
    from sklearn.metrics import r2_score
    r2=r2_score(Y_test,Y_pred)
    return r2

In [7]:
from sklearn.linear_model import LinearRegression
def Linear(X_train,Y_train,X_test):
    regressor=LinearRegression()
    regressor.fit(X_train,Y_train)
    r2=r2_prediction(regressor,X_test,Y_test)
    return r2

from sklearn import svm
def svrl(X_train,Y_train,X_test):
    regressor=svm.SVR(kernel='linear')
    regressor.fit(X_train,Y_train)
    r2=r2_prediction(regressor,X_test,Y_test)
    return r2

def svrnl(X_train,Y_train,X_test):
    regressor=svm.SVR(kernel='rbf')
    regressor.fit(X_train,Y_train)
    r2=r2_prediction(regressor,X_test,Y_test)
    return r2

from sklearn.tree import DecisionTreeRegressor
def decisiontree(X_train,Y_train,X_test):
    regressor=DecisionTreeRegressor(random_state=0)
    regressor.fit(X_train,Y_train)
    r2=r2_prediction(regressor,X_test,Y_test)
    return r2

from sklearn.ensemble import RandomForestRegressor
def randomforest(X_train,Y_train,X_test):
    regressor=RandomForestRegressor(n_estimators=10,random_state=0)
    regressor.fit(X_train,Y_train)
    r2=r2_prediction(regressor,X_test,Y_test)
    return r2

In [32]:
def table(acclin,accsvml,accsvmnl,accdt,accrf):
    dataframe=pd.DataFrame(index=['ChiSquare'],columns=['Linear','SVML','SVMNL','DecisionTree','RandomForest'])
    
    for number,index in enumerate(dataframe.index):
        dataframe['Linear'][index]=acclin[number]
        dataframe['SVML'][index]=accsvml[number]
        dataframe['SVMNL'][index]=accsvmnl[number]
        dataframe['DecisionTree'][index]=accdt[number]
        dataframe['RandomForest'][index]=accrf[number]
    return dataframe

In [81]:
kbest=selectk(independent,dependent,5)
kbest

array([[1.48112676e+02, 5.74821053e+01, 3.07735602e+00, 3.88689024e+01,
        8.40819113e+03],
       [1.48112676e+02, 2.20000000e+01, 7.00000000e-01, 3.40000000e+01,
        1.23000000e+04],
       [9.90000000e+01, 2.30000000e+01, 6.00000000e-01, 3.40000000e+01,
        8.40819113e+03],
       ...,
       [1.10000000e+02, 1.15000000e+02, 6.00000000e+00, 2.60000000e+01,
        9.20000000e+03],
       [2.07000000e+02, 8.00000000e+01, 6.80000000e+00, 3.88689024e+01,
        8.40819113e+03],
       [1.00000000e+02, 4.90000000e+01, 1.00000000e+00, 5.30000000e+01,
        8.50000000e+03]])

In [82]:
X_train,X_test,Y_train,Y_test=split_scaler(kbest,dependent)

In [83]:
acclin=[]
accsvml=[]
accsvmnl=[]
accdt=[]
accrf=[]

In [84]:
r2_lin=Linear(X_train,Y_train,X_test)
acclin.append(r2_lin)
        
r2_svml=svrl(X_train,Y_train,X_test)
accsvml.append(r2_svml)
        
r2_svmnl=svrnl(X_train,Y_train,X_test)
accsvmnl.append(r2_svmnl)
        
r2_dt=decisiontree(X_train,Y_train,X_test)
accdt.append(r2_dt)
        
r2_rf=randomforest(X_train,Y_train,X_test)
accrf.append(r2_rf)

In [85]:
result=table(acclin,accsvml,accsvmnl,accdt,accrf)

C:\Users\DELL\AppData\Local\Temp\ipykernel_11128\3233601580.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['Linear'][index]=acclin[number]
C:\Users\DELL\AppData\Local\Temp\ipykernel_11128\3233601580.py:6: FutureWarning: Chaine

In [68]:
result #10

,Linear,SVML,SVMNL,DecisionTree,RandomForest
ChiSquare,0.644735,0.597726,0.919312,0.869792,0.922743


In [74]:
result #9

,Linear,SVML,SVMNL,DecisionTree,RandomForest
ChiSquare,0.646123,0.602462,0.901819,0.869792,0.919705


In [80]:
result #8

,Linear,SVML,SVMNL,DecisionTree,RandomForest
ChiSquare,0.646457,0.612199,0.891274,0.869792,0.898872


In [62]:
result #7

,Linear,SVML,SVMNL,DecisionTree,RandomForest
ChiSquare,0.657035,0.641906,0.893007,0.826389,0.916233


In [45]:
result #6

,Linear,SVML,SVMNL,DecisionTree,RandomForest
ChiSquare,0.599041,0.586446,0.838962,0.869792,0.897569


In [86]:
result #5

,Linear,SVML,SVMNL,DecisionTree,RandomForest
ChiSquare,0.551985,0.545395,0.749654,0.696181,0.836806


In [56]:
result #4

,Linear,SVML,SVMNL,DecisionTree,RandomForest
ChiSquare,0.304963,0.256858,0.430795,0.479167,0.599392


In [ ]:
#Random Forest algorithm works best for all the values of k, specified here, with a maximum accuracy of 92% for k=10.